In [182]:
import re
import os

In [183]:
def init(input):
    list_fp = []
    list_name = []
    if '.' in input:
        if re.search('.vm$',input):
            try:
                list_fp.append(open(input,'r'))
                list_name.append(input.split('.')[0])
            except IOError as e:
                print('No VM file found!')
    else:
        try:
            dirList = os.listdir(input)
            for each in dirList:
                if re.search('.vm$',each):
                    list_fp.append(open(input + '/' + each,'r'))
                    list_name.append((input + '/' + each).split('.')[0])
        except IOError as e:
            print('No VM file found!')
    return list_fp,list_name

In [188]:
class Parser:
    def __init__(self,fp):
        self.fp = fp
        self.flag = True
        self.count = 0
        self.algorithm_list = ['add','sub','neg','eq','gt','lt','and','or','not']
        
    def hasMoreCommands(self):
        self.line = self.fp.readline()
        if self.line :return True
        else:
            self.flag = False
            return False

    def advance(self):
        if self.hasMoreCommands():
            self.cur = self.line.strip().split('//')[0].strip()
            self.segments = self.cur.split()
            while len(self.cur)== 0 or self.cur[0] == '//': self.advance()
                
    def commandType(self):
        judge = [each for each in self.algorithm_list if each in self.segments]
        if len(judge): return 'C_ARITHMETIC'
        if 'push' in self.segments: return 'C_PUSH'
        if 'pop' in self.segments: return 'C_POP'
        if 'label' in self.segments: return 'C_LABEL'
        if 'if-goto' in self.segments: return 'C_IF'
        if 'goto'  in self.segments: return 'C_GOTO'
        if 'function' in self.segments: return 'C_FUNCTION'
        if 'return' in self.segments: return 'C_RETURN'
        if 'call' in self.segments: return 'C_CALL'
    
    def arg1(self):
        if self.commandType() == 'C_ARITHMETIC': return self.cur
        if self.commandType() == 'C_PUSH': return self.segments[1]
        if self.commandType() == 'C_POP': return self.segments[1]
        if self.commandType() == 'C_LABEL': return self.segments[1]
        if self.commandType() == 'C_IF': return self.segments[1]
        if self.commandType() == 'C_GOTO': return self.segments[1]
        if self.commandType() == 'C_FUNCTION': return self.segments[1]
        if self.commandType() == 'C_CALL': return self.segments[1]
    
    def arg2(self):
        if self.commandType() == 'C_PUSH': return self.segments[2]
        if self.commandType() == 'C_POP': return self.segments[2]
        if self.commandType() == 'C_FUNCTION': return self.segments[2]
        if self.commandType() == 'C_CALL': return self.segments[2]    

In [194]:
class CodeWriter:
    def __init__(self,name):
        self.fw = open(name + '.asm', 'w')
        self.count = 0
        nameList = name.split('/')
        if len(nameList) == 1:self.name = nameList[0]
        else:self.name = nameList[0] + '_' + nameList[1]
        
    '''
    def setFileName(filename)
        pass
    '''
    def writeInit(self):
        self.fw.write('SP=256\n')
        self.writeCall('sys.init',0)
    
    def writeArithmetic(self,Command):
        if Command == 'add': self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@SP\nM=M-1\nA=M\nM=D+M\n@SP\nM=M+1\n')
        if Command == 'sub': self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@SP\nM=M-1\nA=M\nM=M-D\n@SP\nM=M+1\n')
        if Command == 'neg': self.fw.write('@SP\nM=M-1\nA=M\nM=-M\n@SP\nM=M+1\n')
        if Command == 'and': self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@SP\nM=M-1\nA=M\nM=D&M\n@SP\nM=M+1\n')
        if Command == 'or': self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@SP\nM=M-1\nA=M\nM=D|M\n@SP\nM=M+1\n')
        if Command == 'not': self.fw.write('@SP\nM=M-1\nA=M\nM=!M\n@SP\nM=M+1')
        if Command == 'eq': 
            self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@SP\nM=M-1\nA=M\nD=M-D\n@EQ' + str(self.count) \
                                          + '\nD;JEQ\n@SP\nA=M\nM=0\n@CONTINUE' + str(self.count) + '\n0;JMP\n(EQ' + str(self.count) \
                                          + ')\n@SP\nA=M\nM=-1\n(CONTINUE' + str(self.count) + ')\n@SP\nM=M+1\n' )
            self.count += 1
        if Command == 'gt': 
            self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@SP\nM=M-1\nA=M\nD=M-D\n@GT' + str(self.count) \
                                          + '\nD;JGT\n@SP\nA=M\nM=0\n@CONTINUE' + str(self.count) + '\n0;JMP\n(GT' + str(self.count) \
                                          + ')\n@SP\nA=M\nM=-1\n(CONTINUE' + str(self.count) + ')\n@SP\nM=M+1\n' )
            self.count += 1
        if Command == 'lt': 
            self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@SP\nM=M-1\nA=M\nD=M-D\n@LT' + str(self.count) \
                                          + '\nD;JLT\n@SP\nA=M\nM=0\n@CONTINUE' + str(self.count) + '\n0;JMP\n(LT' + str(self.count) \
                                          + ')\n@SP\nA=M\nM=-1\n(CONTINUE' + str(self.count) + ')\n@SP\nM=M+1\n' )
            self.count += 1
    
    def writePushPop(self,Command,segment,index):
        if segment == 'constant':
            if Command == 'C_PUSH':self.fw.write('@' + index + '\nD=A\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
                
        if segment == 'argument':
            if Command == 'C_PUSH':self.fw.write('@' + index + '\nD=A\n@ARG\nA=M+D\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
            if Command == 'C_POP':
                self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@ARG\nA=M\n')
                for i in range(int(index)):
                    self.fw.write('A=A+1\n')
                self.fw.write('M=M+D\n')
        if segment == 'local':
            if Command == 'C_PUSH':self.fw.write('@' + index + '\nD=A\n@LCL\nA=M+D\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
            if Command == 'C_POP':
                self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@LCL\nA=M\n')
                for i in range(int(index)):
                    self.fw.write('A=A+1\n')
                self.fw.write('M=D\n')        
        if segment == 'temp':
            if Command == 'C_PUSH':self.fw.write('@' + index + '\nD=A\n@R5\nA=M+D\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
            if Command == 'C_POP':
                self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@R5\nA=M\n')
                for i in range(int(index)):
                    self.fw.write('A=A+1\n')
                self.fw.write('M=D\n')        
        if segment == 'this':
            if Command == 'C_PUSH':self.fw.write('@' + index + '\nD=A\n@THIS\nA=M+D\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
            if Command == 'C_POP':
                self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@THIS\nA=M\n')
                for i in range(int(index)):
                    self.fw.write('A=A+1\n')
                self.fw.write('M=D\n')
        if segment == 'that':
            if Command == 'C_PUSH':self.fw.write('@' + index + '\nD=A\n@THAT\nA=M+D\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
            if Command == 'C_POP':
                self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@THAT\nA=M\n')
                for i in range(int(index)):
                    self.fw.write('A=A+1\n')
                self.fw.write('M=D\n')
        if segment == 'pointer':
            if Command == 'C_PUSH':
                if index == '0': self.fw.write('@THIS\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
                if index == '1': self.fw.write('@THAT\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
            if Command == 'C_POP':
                if index == '0': self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@THIS\nM=D\n')
                if index == '1': self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@THAT\nM=D\n')
        if segment == 'static':
            if Command == 'C_PUSH':
                self.fw.write('@' + self.name + '.' + index + '\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
            if Command == 'C_POP':
                self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@' + self.name + '.' + index + '\nM=D\n')
                
    def writeLabel(self,label):
        self.fw.write('(' + label + ')\n')
    
    def wirteGoto(self,label):
        self.fw.write('@' + label + '\n0;JMP\n')
        
    def writeIf(self,label):
        self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@' + label + '\nD;JNE\n')
    
    def writeCall(self,functionName,numArgs):
        self.fw.write('@returnAddress' + str(self.count) + '\nD=A\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
        self.fw.write('@LCL\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
        self.fw.write('@ARG\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
        self.fw.write('@THIS\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
        self.fw.write('@THAT\nD=M\n@SP\nA=M\nM=D\n@SP\nM=M+1\n')
        self.fw.write('@' + str(int(numArgs) + 5) + '\nD=A\n@SP\nD=M-D\n@ARG\nM=D\n')
        self.fw.write('@SP\nD=M\n@LCL\nM=D\n')
        self.fw.write('@' + functionName + '\n0;JMP')
        self.fw.write('(retrunAddress' + self.count + ')\n')
        self.count += 1
    
    def writeReturn():
        self.fw.write('@LCL\nD=M\n@FRAME\nM=D\n')
        self.fw.write('@5\nD=M\n@FRAME\nA=M-D\nD=M\n@RET\nM=D\n')
        self.fw.write('@SP\nM=M-1\nA=M\nD=M\n@ARG\nA=M\nM=D\n')
        self.fw.wrtie('@ARG\nD=M\n@SP\nM=D+1\n')
        self.fw.write('@FRAME\nD=M\n@1\nA=D-A\nD=M\n@THAT\nM=D\n')
        self.fw.write('@FRAME\nD=M\n@2\nA=D-A\nD=M\n@THIS\nM=D\n')
        self.fw.write('@FRAME\nD=M\n@3\nA=D-A\nD=M\n@ARG\nM=D\n')
        self.fw.write('@FRAME\nD=M\n@4\nA=D-A\nD=M\n@LCL\nM=D\n')
        self.fw.write('@RET\n0;JMP\n')
    
    def writeFunction(self,functionName,numLocals):
        self.write('(' + functionName + ')\n')
        for i in range(int(numLocals)):
            self.write('C_PUSH','constant','0')
            
    
    def close(self):
        self.fw.close()
    

In [3]:
def main():
    instruction = input()
    source = instruction.split()[-1]
    if re.match('$Prompt> Vmtranslator',instruction):
        list_fp,list_name = init(source)
        for i in range(len(list_fp)):
            mypsr = Parser(list_fp[i])
            mycoder = CodeWriter(list_name[i])
            print(mycoder.name)
            while mypsr.flag:
                mypsr.advance()
                if mypsr.flag:
                    if mypsr.commandType() == 'C_ARITHMETIC': mycoder.writeArithmetic(mypsr.arg1())
                    if mypsr.commandType() == 'C_PUSH' or 'C_POP': mycoder.writePushPop(mypsr.commandType(),mypsr.arg1(),mypsr.arg2())
            mypsr.fp.close()
            mycoder.close()    